# Tarea Semanal 8

<img src="./imagenes/logo_UTN.svg"  width="300" /> 



#### Por Elian Andrenacci

#### Ejercicio 1 
<img src="./imagenes/Enunciado_1.png" width="auto" />

### Filtro FIR

Para este filtro se prueban todos los tipos de ventana de la libreria firwin2, se elije la ventana hamming ya que nos da el mejor resultado (se utilizan 3001 coeficientes). 

Para graficar la fase se utiliza la herramienta pyfda, en la que se puede ver que esta es lineal 

<img src="./imagenes/P1_FIR_modulo_a.png"/>
<img src="./imagenes/P1_FIR_modulo_b.png"/>
<img src="./imagenes/P1_FIR_fase_a.png"/>
<img src="./imagenes/P1_FIR_retGrupo_b.png"/>


### Filtro IIR

Para generar este tipo de filtro se utiliza la funcion iirdesign, por errores numericos se deben armar sistemas de segundo orden y graficar el filtro
a partir de la SOS.

La fase en los filtros IIR ya no es lineal y se deben utilizar algunas tecnicas para procesar señales. 

<img src="./imagenes/P1_IIR_modulo_a.png" width="auto" />
<img src="./imagenes/P1_IIR_modulo_b.png" width="auto" />
<img src="./imagenes/P1_IIR_fase.png" width="auto" />
<img src="./imagenes/P1_IIR_retGrupo.png" width="auto" />


In [ ]:
## Filtro FIR
import numpy as np
import scipy.signal as sig
import matplotlib as mpl
import matplotlib.pyplot as plt

from pytc2.sistemas_lineales import plot_plantilla



# Parametros de plantilla

ripple = 0.5 # dB
atenuacion = 40 # dB

ws1 = 1.0 #Hz
wp1 = 3.0 #Hz
wp2 = 25.0 #Hz
ws2 = 35.0 #Hz

fs = 1000 # Hz
nyq_frec = fs / 2

frecs = np.array([0.0,         ws1,         wp1,     wp2,     ws2,         nyq_frec   ]) / nyq_frec
gains = np.array([-atenuacion, -atenuacion, -ripple, -ripple, -atenuacion, -atenuacion])
gains = 10**(gains/20)

cant_coeficientes = 3001


num = sig.firwin2(cant_coeficientes, frecs, gains, window='hamming')
den = 1.0


w  = np.append(np.logspace(-1, 0.8, 250), np.logspace(0.9, 1.6, 250) )
w  = np.append(w, np.linspace(110, nyq_frec, 100, endpoint=True) ) / nyq_frec * np.pi


_, h = sig.freqz(num, den, w)


# renormalizo el eje de frecuencia
w = w / np.pi * nyq_frec

plt.plot(w, 20 * np.log10(abs(h)), label='FIR ventana hamming')


plt.title('Filtros diseñados')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Módulo [dB]')
plt.grid()
plt.axis([0, 100, -60, 5 ])

axes_hdl = plt.gca()
axes_hdl.legend()

plot_plantilla(filter_type = 'bandpass', fpass = frecs[[2, 3]]* nyq_frec, ripple = ripple , fstop = frecs[ [1, 4] ]* nyq_frec, attenuation = atenuacion, fs = fs)


In [ ]:
## Filtro IIR
import numpy as np
import scipy.signal as sig
import matplotlib as mpl
import matplotlib.pyplot as plt

from pytc2.sistemas_lineales import plot_plantilla, analyze_sys



ws1 = 0.01 #Hz
wp1 = 0.03 #Hz
wp2 = 0.25 #Hz
ws2 = 0.35 #Hz


ripple = 0.5        # dB
attenuation = 40    # dB

fs = 2



sos = sig.iirdesign([wp1, wp2], [ws1, ws2], ripple, attenuation, analog=False, ftype='butter', output='sos')
w, h = sig.sosfreqz(sos, fs=fs)



plt.figure(1)
plt.cla()

plt.plot(w, 20 * np.log10(np.abs(h)), label='Filtro digital')
plt.plot(w, np.angle(h), label='Filtro digital')

plt.title('Plantilla de diseño')
plt.xlabel('Frecuencia normalizada a Nyq [#]')
plt.ylabel('Amplitud [dB]')
plt.grid(which='both', axis='both')

plt.gca().set_xlim([0, 1])

plot_plantilla(filter_type = "bandpass" , fpass = [wp1, wp2], ripple = ripple , fstop = [ws1, ws2], attenuation = attenuation, fs = fs)


### Parte 2

Filtro FIR

Ahora se ingresa con una señal de un ECG y se utiliza el filtro pasabanda antes diseñado.

<img src="./imagenes/P2_FIR_filtradoECG_a.png" width="auto" />


Filtro IIR

Para el filtro IIR se utiliza una tecnica de filtrado bidireccional que evita por completo cualquier distorcion de fase. Con esto se obtiene un filtro de fase cero. Por otro lado, se debe modificar la plantilla de diseño relajando a la mitad a la atenuacion pero duplicando la exigencia de ripple

<img src="./imagenes/P2_IIR_filtradoECG_a.png" width="auto" />


Ambos filtros son muy fiables salvando las oscilaciones que introducen, esto se debe a los pulsos complejos QRS se asemejan a una delta. Por otro lado, la tecnica de filtrado direccional con la que obtenemos filtros de fase cero no se puede hacer en tiempo real pero una vez grabada la señal la podemos aplicar en ambos tipos de filtros. 

In [ ]:
## Filtro FIR
import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt
import scipy.io as sio

from pytc2.sistemas_lineales import plot_plantilla

mat_struct = sio.loadmat('ecg.mat')
ecg_one_lead = mat_struct['ecg_lead']
ecg_one_lead = ecg_one_lead.flatten()
cant_muestras = len(ecg_one_lead)

# Parametros de plantilla

ripple = 0.5 # dB
atenuacion = 40 # dB

ws1 = 1.0 #Hz
wp1 = 3.0 #Hz
wp2 = 25.0 #Hz
ws2 = 35.0 #Hz

fs = 1000 # Hz
nyq_frec = fs / 2

# Configuracion de grafico
fig_sz_x = 10
fig_sz_y = 7
fig_dpi = 100 # dpi

frecs = np.array([0.0,         ws1,         wp1,     wp2,     ws2,         nyq_frec   ]) / nyq_frec
gains = np.array([-atenuacion, -atenuacion, -ripple, -ripple, -atenuacion, -atenuacion])
gains = 10**(gains/20)

cant_coeficientes = 3001


num = sig.firwin2(cant_coeficientes, frecs, gains, window='hamming')
den = 1.0


ECG_f_win = sig.lfilter(num, den, ecg_one_lead)

demora = 750

regs_interes = ( 
        np.array([5.148, 5.16]) *60*fs, # minutos a muestras
        np.array([5, 5.2]) *60*fs, # minutos a muestras
        np.array([12, 12.4]) *60*fs, # minutos a muestras
        np.array([15, 15.2]) *60*fs, # minutos a muestras
)

for ii in regs_interes:
    
    # intervalo limitado de 0 a cant_muestras
    zoom_region = np.arange(np.max([0, ii[0]]), np.min([cant_muestras, ii[1]]), dtype='uint')
    
    plt.figure(dpi= fig_dpi, facecolor='w', edgecolor='k')
    plt.plot(zoom_region, ecg_one_lead[zoom_region], label='ECG', linewidth=1.5)
    #plt.plot(zoom_region, ECG_f_butt[zoom_region], label='Butter')
    plt.plot(zoom_region, ECG_f_win[zoom_region + demora], label='Win')
    
    plt.title('ECG filtering example from ' + str(ii[0]) + ' to ' + str(ii[1]) )
    plt.ylabel('Adimensional')
    plt.xlabel('Muestras (#)')
    
    axes_hdl = plt.gca()
    axes_hdl.legend()
    axes_hdl.set_yticks(())
            
    plt.show()

In [ ]:
## Filtro IIR
import numpy as np
import scipy.signal as sig
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.io as sio

from pytc2.sistemas_lineales import plot_plantilla

mat_struct = sio.loadmat('ecg.mat')
ecg_one_lead = mat_struct['ecg_lead']
ecg_one_lead = ecg_one_lead.flatten()
cant_muestras = len(ecg_one_lead)

fig_dpi = 100 # dpi

ws1 = 0.1   #Hz
wp1 = 1     #Hz
wp2 = 25    #Hz
ws2 = 35    #Hz


ripple = 0.1        # dB
attenuation = 20    # dB

fs = 1000 # Hz
nyq_frec = fs / 2


sos = sig.iirdesign(np.array([wp1, wp2]) / nyq_frec, np.array([ws1, ws2]) / nyq_frec, gpass=ripple, gstop=attenuation, analog=False, ftype='butter', output='sos')


ECG_f_win = sig.sosfiltfilt(sos, ecg_one_lead)

# Segmentos de interés
regs_interes = ( 
    [4000, 5500], # muestras
    [10e3, 11e3], # muestras
)

for ii in regs_interes:
    
    # intervalo limitado de 0 a cant_muestras
    zoom_region = np.arange(np.max([0, ii[0]]), np.min([cant_muestras, ii[1]]), dtype='uint')
    
    plt.figure(dpi= fig_dpi, facecolor='w', edgecolor='k')
    plt.plot(zoom_region, ecg_one_lead[zoom_region], label='ECG original', lw=2)
    plt.plot(zoom_region, ECG_f_win[zoom_region], label='ECG filtrado')
    
    plt.title('ECG muestras de ' + str(ii[0]) + ' a ' + str(ii[1]) )
    plt.ylabel('Adimensional')
    plt.xlabel('Muestras (#)')
    
    axes_hdl = plt.gca()
    axes_hdl.legend()
    axes_hdl.set_yticks(())
            
    plt.show()